# 협업 필터링 : 사용자 리뷰 기반

유저에 따라 같은 영화여도 예측 평가 점수가 달라진다

In [7]:
import surprise
surprise.__version__

'1.1.4'

In [9]:
import pandas as pd
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate

In [11]:
ratings = pd.read_csv('ratings_small.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [13]:
ratings['rating'].min()

0.5

In [15]:
ratings['rating'].max()

5.0

In [17]:
reader = Reader(rating_scale=(0.5, 5))

In [21]:
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader=reader)
data

In [23]:
svd = SVD(random_state=0)

In [25]:
cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=5, verbose=True) 

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8896  0.9040  0.8924  0.8990  0.8949  0.8960  0.0051  
MAE (testset)     0.6862  0.6921  0.6873  0.6925  0.6882  0.6892  0.0026  
Fit time          1.65    1.82    1.59    1.68    1.86    1.72    0.10    
Test time         0.34    0.38    0.56    0.29    0.37    0.39    0.09    


{'test_rmse': array([0.8895985 , 0.90404016, 0.89239555, 0.89902745, 0.89487358]),
 'test_mae': array([0.68617414, 0.6920505 , 0.68730141, 0.69248915, 0.68822383]),
 'fit_time': (1.6484558582305908,
  1.8187966346740723,
  1.5859153270721436,
  1.6788454055786133,
  1.8579916954040527),
 'test_time': (0.3362574577331543,
  0.38121819496154785,
  0.5610091686248779,
  0.2886974811553955,
  0.37255430221557617)}

교차 검증 (K-Fold 교차 검증)  
  
100개의 데이터가 있을 때  
  
A : 1-20  
B : 21-40  
C : 41-60  
D : 61-80  
E : 81-100  

ABCD (train set) E (test set)  
ABCE (train set) D (test set)  
ABDE (train set) C (test set)  
ACDE (train set) B (test set)  
BCDE (train set) A (test set)  

In [29]:
trainset = data.build_full_trainset()
svd.fit(trainset)

In [31]:
ratings[ratings['userId'] == 1]

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
5,1,1263,2.0,1260759151
6,1,1287,2.0,1260759187
7,1,1293,2.0,1260759148
8,1,1339,3.5,1260759125
9,1,1343,2.0,1260759131


In [39]:
svd.predict(1, 302) # userId가 1인 사람이 movieId가 302인 영화 평가점수의 예상 값

Prediction(uid=1, iid=302, r_ui=None, est=2.7142061734434044, details={'was_impossible': False})

In [43]:
svd.predict(1, 1029) # userId가 1인 사람이 movieId가 1029인 영화에 대해서 실제 평가 3점일때 예측 평가 점수

Prediction(uid=1, iid=1029, r_ui=None, est=2.8814455446761933, details={'was_impossible': False})

In [45]:
ratings[ratings['userId'] == 100]

,userId,movieId,rating,timestamp
15273,100,1,4.0,854193977
15274,100,3,4.0,854194024
15275,100,6,3.0,854194023
15276,100,7,3.0,854194024
15277,100,25,4.0,854193977
15278,100,32,5.0,854193977
15279,100,52,3.0,854194056
15280,100,62,3.0,854193977
15281,100,86,3.0,854194208
15282,100,88,2.0,854194208


In [49]:
svd.predict(100, 1029) # userId = 100, movieId = 1029

Prediction(uid=100, iid=1029, r_ui=None, est=3.7705476478414846, details={'was_impossible': False})